In [ ]:
# TASK 1


from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import datetime as dt
import time
import json
import pandas as pd
try:
    import urlparse
except ImportError:
    import urllib.parse as urlparse
import re



ACCESS_TOKEN = "1078653960314122240-5Bk1L531cMZ9dtB39YTWHxdqbrMXcU"
ACCESS_TOKEN_SECRET = "k9ZS7sxbVINDwgUAB6qCMHmOxPUNbrrxKzAQHgYwRzDUc"
CONSUMER_KEY = "GSaVJHUfTRVKrD5pzL2kGeKq8"
CONSUMER_SECRET = "rtaMUnyrpJLBA1zSv9wtXnxTCNwdlN3f1lqmrv3UHpaDQv4dd7"


class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        pass

    def stream_tweets(self, fetched_tweets_filename, fetched_links_filename,hash_tag_list,fetched_text_filename):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(languages=["en"],track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class StdOutListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename,time_limit=15):
        self.fetched_tweets_filename = fetched_tweets_filename
        self.fetched_links_filename = fetched_links_filename
        self.fetched_text_filename = fetched_text_filename
        self.start_time = time.time()
        self.limit = time_limit
        super(StdOutListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            try:
                with open(self.fetched_tweets_filename, 'a') as tf,open(self.fetched_links_filename,'a') as lf,open(self.fetched_text_filename,'a') as wf:
                    for url in status.entities['urls']:
                        lf.write(str(url['expanded_url']+" "))
                    
                    wf.write(str(status.text))
                    tf.write(str(status.user.screen_name+" "))
                return True
            except BaseException as e:
                print("Error on_status %s" % str(e))
            return True
        else:
            return False
          

    def on_error(self, status):
        print(status)
        
        
class xyz:
    
    def __init__(self):
        pass
    
    def no_of_tweets(self):
        f = open("task1_tweets.txt", "r")
        x = f.read()
        lst = x.split(" ")
        y = dict((x,lst.count(x)) for x in set(lst))
        del y['']
        user_name = []
        No_of_tweets = []
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', None)
        for i in y : 
            user_name.append(i)
            No_of_tweets.append(int(y[i]))
        df = pd.DataFrame({'user_name':user_name ,
                           'No_of_tweets': No_of_tweets})
        return df
    
    def links(self):
        f = open("task2_links.txt", "r")
        x = f.read()
        lst = x.split(" ")
        link_list = []

        for string in lst:
            parsed = urlparse.urlparse(string)
            domain = parsed.netloc.split(".")[0:]
            host = ".".join(domain)
            link_list.append(host)
        link_list.remove("")

        y = dict((x,link_list.count(x)) for x in set(link_list))
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', None)
        domain , no_of_domain= [],[]

        for i in y : 
            domain.append(i)
            no_of_domain.append(int(y[i]))
        df = pd.DataFrame({'domain':domain ,
                           'number_of_occurence': no_of_domain})
        df = df.sort_values(by='number_of_occurence', ascending=False)
        print("Total number of links is :",len(link_list))
        return df    

    def words(self):
        common_words = ['can','her','was','will','dont','your','if','its','from','like','are','but','we','and','you','for','this','with','me','not','be','is','so','im','by','to','on','of', 'at', 'in', 'without', 'between', 'he', 'they', 'anybody', 'it', 'one','the', 'a', 'that', 'my', 'more', 'much', 'either', 'neither','i','have', 'got', 'do','no','that', 'when','']
        f = open("task1_words.txt", "r")
        x = f.read()
        x = " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", x).split()) # here we are removing emojis and extra garbage which isnt word
        words_list = x.split(" ")
        lower_case = [word.lower() for word in words_list]

        new_words = [word for word in lower_case if word not in common_words]
        y = dict((x,new_words.count(x)) for x in set(new_words))
        word = []
        count_of_words = []
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', None)
        for i in y : 
            word.append(i)
            count_of_words.append(int(y[i]))
        df = pd.DataFrame({'word':word ,
                           'count_of_words': count_of_words})
        df = df.sort_values(by='count_of_words', ascending=False)
        df = df.head(10)
        return df
    
    def erase(self):
        open("task1_tweets.txt", "w").close()
        open("task1_links.txt", "w").close()
        open("task1_words.txt", "w").close()
        

 
if __name__ == '__main__':
 
    # Authenticate using config.py and connect to Twitter Streaming API.
    
    keyword = input(" please Enter keyword you want to search: ")
    hash_tag_list = [keyword]
    fetched_tweets_filename = "task1_tweets.txt"
    fetched_links_filename = "task1_links.txt"
    fetched_text_filename = "task1_words.txt"

    
    while True:
        twitter_streamer = TwitterStreamer()
        obj1 = xyz()
        twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list,fetched_links_filename,fetched_text_filename)
        print("User report \n",obj1.no_of_tweets())
        print("Links report\n",obj1.links())
        print("Word report\n",obj1.words())
        obj1.erase()  # delete content after one minute
         
